In [1]:
# imports
import json
from pathlib import Path
import os
import sys
import pickle
import torch
from tqdm import tqdm

# Paths preprocessed datasets, generated by utils/prep.py (to speed up training)
DIR_DATASET = Path(current_dir=os.getcwd()).parent.absolute()/'dataset_DocRED'
DATA_DEV = DIR_DATASET/'dev.json' 
DATA_TRAIN = DIR_DATASET/'train_annotated.json'

DIR_DATASET_PREP = Path(current_dir=os.getcwd()).parent.absolute()/'dataset_prep'
DATA_DEV_PREP = DIR_DATASET_PREP/'dev_prep.pickle' 
DATA_TRAIN_PREP = DIR_DATASET_PREP/'train_annotated_prep.pickle'

PATH_UTILS = DIR_DATASET.parent.absolute().__str__()

# add local lib to path
if PATH_UTILS not in sys.path:
    sys.path.append(PATH_UTILS)

C:\Users\Robin\AppData\Local\Temp\ipykernel_37128\24878954.py:11: DeprecationWarning: support for supplying keyword arguments to pathlib.PurePath is deprecated and scheduled for removal in Python 3.14
  DIR_DATASET = Path(current_dir=os.getcwd()).parent.absolute()/'dataset_DocRED'
C:\Users\Robin\AppData\Local\Temp\ipykernel_37128\24878954.py:15: DeprecationWarning: support for supplying keyword arguments to pathlib.PurePath is deprecated and scheduled for removal in Python 3.14
  DIR_DATASET_PREP = Path(current_dir=os.getcwd()).parent.absolute()/'dataset_prep'


In [2]:
from utils import DocREDDataset, collate_fn
from torch.utils.data import DataLoader, random_split
# load preped dataset
fData = open(DATA_TRAIN_PREP.__str__(), 'rb')
data = pickle.load(fData)
fData.close()

# train/val split
train_size = int(0.8 * len(data))  # 80% for training
val_size = len(data) - train_size  # 20% for validation

# test dataset
fData = open(DATA_DEV_PREP.__str__(), 'rb')
data_test = pickle.load(fData)
fData.close()

In [ ]:
# Define model
from torch import nn
from pytorch_tcn import TemporalConv1d

# TCN block
class TCN_Block(nn.Module):
    def __init__(self, in_features, out_features, kernel_size=7, stride=1, dilation=1, dropout=0.3):
        super(TCN_Block, self).__init__()

        # temporal conv with weight_norm
        self.res = nn.Conv1d(in_features, out_features, kernel_size=1) # 1x1 conv sampling for residual
        self.conv = nn.utils.parametrizations.weight_norm(TemporalConv1d(in_features, out_features, kernel_size, stride, dilation=dilation))
        self.relu = nn.ReLU() 
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.conv(x) # Dilated Causal Conv
        x = self.dropout(self.relu(x)) # dropout regularization
        return x

# The model
class LSTM_TCNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, hidden_dims_tcn, dropout=0.3):
        super(LSTM_TCNClassifier, self).__init__()
        self.coref_embed = nn.Embedding(512, 20, padding_idx=0)
        self.ner_embed = nn.Embedding(512, 20, padding_idx=0)

        # tcn
        tcn_layers = []
        for i in range(len(hidden_dims_tcn)):
            in_features = 0
            if i == 0:
                in_features = input_dim
            else:
                in_features = hidden_dims_tcn[i-1]
            out_features = hidden_dims_tcn[i]
            dilation = 2 ** i # dilation 1, 2, 4, ...
            tcn_layer = TCN_Block(in_features, out_features, dilation=dilation, dropout=dropout)
            tcn_layers.append(tcn_layer)
        self.tcn = nn.Sequential(*tcn_layers)

        # lstm
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        # layer norms
        self.ln_h = nn.LayerNorm(hidden_dim)
        self.ln_t = nn.LayerNorm(hidden_dim)

        # classification
        self.dropout1 = nn.Dropout(p=dropout)
        self.fc1 = nn.Linear(hidden_dim+hidden_dims_tcn[-1], hidden_dim)
        self.relu1 = nn.ReLU()

        # bilinear output
        self.bln = nn.Bilinear(hidden_dim*2, hidden_dim*2, output_dim)

        # self.softmax = nn.Softmax(dim=-1) # NOTE: removed, output bilinear, use sigmoid later

        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
    
    def forward(self, embed, coref, ner, map_h, map_t):
        # coref 
        coref = self.coref_embed(coref)
        # ner (coref_type)
        ner = self.ner_embed(ner)
        
        net_in = torch.cat([embed, coref, ner], dim=-1)

        # tcn
        tcn_out = self.tcn(net_in.transpose(-1, -2)).transpose(-1, -2)

        # lstm
        hidden, carry = torch.zeros(self.num_layers, net_in.shape[0], self.hidden_dim).to(embed.device), torch.zeros(self.num_layers, net_in.shape[0], self.hidden_dim).to(embed.device)
        # xavier init hidden and carry for stability over randn
        nn.init.xavier_normal_(hidden)
        nn.init.xavier_normal_(carry)
        lstm_out, (hidden, carry) = self.lstm(net_in, (hidden, carry))


        # Extract features from time sequence output

        # concat tcn and lstm features
        seq_out = torch.cat([tcn_out, lstm_out], dim=-1)
        # linear layer
        seq_out = self.dropout1(self.relu1(self.fc1(seq_out)))

        # saperate features for head and tail entities
        map_h = map_h.float().unsqueeze(1)
        map_t = map_t.float().unsqueeze(1)

        seq_out_h = self.ln_h(torch.matmul(map_h, seq_out).squeeze(1)) 
        seq_out_t = self.ln_t(torch.matmul(map_t, seq_out).squeeze(1))

        dist = seq_out_t - seq_out_h

        seq_out_h = torch.concat([seq_out_h, dist], dim=-1)
        seq_out_t = torch.concat([seq_out_t, -dist], dim=-1)
        
        # bilinear output
        out = self.bln(seq_out_h, seq_out_t)
        return out

# Model Setup & Training

(Note: Training took about 1 hour on a machine with 8C16T CPU and Nvidia RTX 3060-12G, loading the pre-processed training and testing datasets takes around 26GB of RAM)

In [4]:
# Parameters
input_dim = 768 + 20 + 20  # DistilledBERT pretrained embeddings + coref + ner type (both head and tail)
hidden_dim = 128
hidden_dims_tcn=[256,128,64]
dropout = 0.5
output_dim = 96 + 1  # 96 types + Na
num_layers = 2
batch_size = 32
learning_rate = 0.0002
max_epochs = 50 # has to be multiple of k_folds

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

# Split dataset for training and validation
dataset = DocREDDataset(data, na_factor=0.5) # introduce new random NA examples in new k-fold cycle
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# model
model = LSTM_TCNClassifier(input_dim, hidden_dim, output_dim, num_layers, dropout=dropout, hidden_dims_tcn=hidden_dims_tcn).to(device)
criterion = nn.BCEWithLogitsLoss() # Classification loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train(model, embed, coref, ner, y, map_h, map_t):
    optimizer.zero_grad()
    outputs = model(embed, coref, ner, map_h, map_t)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    return loss.item()

import numpy as np
# store train losses
losses_train = []
losses_val = []

# training loop
(DIR_DATASET_PREP.parent/'checkpoints').mkdir(exist_ok=True) # create output directory
for epoch in range(1, max_epochs+1):
    # train
    train_loss = []
    for batch in tqdm(train_loader, desc=f"epoch {epoch} [Train]"):
        x, y = batch

        x = x.to(device)
        y = y.to(device)

        embed = x[:,:,:768]
        coref = x[:,:,768].long()
        ner = x[:,:,769].long()
        map_h = x[:,:,770]
        map_t = x[:,:,771]

        loss = train(model, embed, coref, ner, y, map_h, map_t)
        train_loss.append(loss)
    avg_train_loss = sum(train_loss)/len(train_loss)
    losses_train.append(avg_train_loss)

    # val
    val_loss = []
    for batch in tqdm(val_loader, desc=f"epoch {epoch} [Val]"):
        with torch.no_grad():
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            embed = x[:,:,:768]
            coref = x[:,:,768].long()
            ner = x[:,:,769].long()
            map_h = x[:,:,770]
            map_t = x[:,:,771]
            outputs = model(embed, coref, ner, map_h, map_t)
            loss = criterion(outputs, y)
            val_loss.append(loss.item())
    avg_val_loss = sum(val_loss)/len(val_loss)
    losses_val.append(avg_val_loss)

    print(f"loss_train={avg_train_loss}, loss_val={avg_val_loss}")
    
    if epoch % 5 == 0:
        # save checkpoint 
        torch.save(model.state_dict(), DIR_DATASET_PREP.parent/'checkpoints'/f'lstm_tcn_{epoch}e.model')

# save loss for logging
loss_train = np.array(losses_train)
np.save(DIR_DATASET_PREP.parent/'checkpoints'/'loss_train.npy', loss_train)
loss_val = np.array(losses_val)
np.save(DIR_DATASET_PREP.parent/'checkpoints'/'loss_val.npy', loss_val)

epoch 1 [Val]: 100%|██████████| 354/354 [00:06<00:00, 55.11it/s]


loss_train=0.09923500584679677, loss_val=0.04370540759847157


epoch 2 [Val]: 100%|██████████| 354/354 [00:06<00:00, 54.93it/s]


loss_train=0.0380226759304624, loss_val=0.03458818678591548


epoch 3 [Val]: 100%|██████████| 354/354 [00:06<00:00, 54.28it/s]


...

loss_train=0.007874741356326195, loss_val=0.01985068650821508


epoch 49 [Val]: 100%|██████████| 354/354 [00:06<00:00, 55.88it/s]


loss_train=0.007554659471957189, loss_val=0.019216489612445823


epoch 50 [Val]: 100%|██████████| 354/354 [00:06<00:00, 54.81it/s]

loss_train=0.007452232037573362, loss_val=0.018806043190166016
